In [12]:
import simulate
from PlateSolve3.PlaneWave import platesolve
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from astropy.io import fits
from astropy.table import Table
import astropy.units as u
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.stats import scoreatpercentile, percentileofscore
from os import walk,path,makedirs,listdir
import shutil
%matplotlib tk

NameError: name 'exit' is not defined

In [10]:
im = np.load('./essential_files/kernels_294/r=7009.npy')

In [11]:
plt.imshow(im,norm=LogNorm())

In [3]:
def make_image(ra,dec,bgd,seeing,texp,maxmag,resolution,dirname,filename,kerdir=None):
    params = {'max_mag':maxmag,
          'ra':ra, 'dec': dec,
        #   'op_mode':'Spectra',
          'op_mode':'Image',
          't_exp':texp,
          'bgd':bgd,
          'seeing':seeing,
          'temp':-15,
          'read_rms':3,
          'resolution':resolution,
          'kernel_dir':kerdir
          }
    image = simulate.simulate_image(params)
    filepath = path.join('.','images',dirname,filename) + '.fits'
    simulate.save_as_fits(image,params,filepath)

def create_images_uniformly(N,bgd,seeing,dirname):
    icdf = lambda p: np.rad2deg(np.arccos(1 - 2*p))
    dec_arr = icdf(np.random.random(N)) - 90
    ra_arr = np.random.uniform(0,360,N)
    i_arr = np.arange(1,N+1,1)
    texp,maxmag,resolution = 1,15,1

    for ra,dec,i in tqdm(zip(ra_arr,dec_arr,i_arr),total = len(i_arr)):
        make_image(ra,dec,bgd,seeing,texp,maxmag,resolution,dirname,f'{int(i)}')

def create_precision_images(N,bgd,seeing,resolution,kerdir,base_dir = 'prec'):
    p = simulate.plate_scale_arcsec_px.value/3600

    icdf = lambda p: np.rad2deg(np.arccos(1 - 2*p))
    center_dec = icdf(np.random.random(N)) - 90
    center_ra = np.random.uniform(0,360,N)

    dirnames = listdir(path.join('.','images',base_dir))
    i0 = np.max([len(dirnames), 1])
    i_arr = np.arange(i0,N+1,1)

    # kerdir = simulate.create_kernels(resolution,seeing * u.arcsec)
    texp,maxmag = 1,16

    for ra,dec,i in tqdm(zip(center_ra,center_dec,i_arr),total = len(i_arr)):
        dirname = path.join(base_dir,f'{int(i)}')
        path_check = path.join('.','images',dirname)
        if not path.exists(path_check):
            makedirs(path_check)
            make_image(ra,dec,bgd,seeing,texp,maxmag,resolution,dirname,'cntr',kerdir)

            delta_ra = p * np.cos(dec)
            delta_dec = p
            shift_ra = np.random.uniform(-4,4) * delta_ra
            shift_dec = np.random.uniform(-4,4) * delta_dec
            ra2 = ra + shift_ra
            dec2 = dec + shift_dec
            make_image(ra2,dec2,bgd,seeing,texp,maxmag,resolution,dirname,'shift',kerdir)
    
    # shutil.rmtree(kerdir)
    
def rand_solve(l,h,t_exp):
    i = np.random.randint(low = l,high = h)
    platesolve.platesolve(f'./images/exp{int(t_exp)}/{i}.fits',simulate.plate_scale_arcsec_px.value)

def stats_solve(dirpath,bgd,seeing,resolution):
    
    filenames = next(walk(dirpath), (None, None, []))[2]  # [] if no file
    n = len(filenames)
    ra_img = np.zeros(n)
    dec_img = np.zeros(n)
    arr1 = np.zeros(n)
    arr5 = np.zeros(n)
    arr10 = np.zeros(n)
    for i in tqdm(range(n)):
        hdu = fits.open(path.join(dirpath,filenames[i]))
        ra_img[i] = hdu[0].header['RA']
        dec_img[i] = hdu[0].header['DEC']
        hdu.close()
        ps = platesolve.platesolve(path.join(dirpath,filenames[i]),simulate.plate_scale_arcsec_px.value)
        if len(ps) > 0: ## if image is solved for 1 sec exposure, mark true for 1,5,10 sec.
            arr1[i] = 1
            arr5[i] = 1
            arr10[i] = 1
        else: ## if image isn't solved, check first if it's because I queried too few stars (too many should be covered by simulate.py)
            texp = 1
            make_image(ra_img[i],dec_img[i],bgd,seeing,texp,17,resolution,dirpath.removeprefix('./images/'),filenames[i].removesuffix('.fits'))
            ps2 = platesolve.platesolve(path.join(dirpath,filenames[i]),simulate.plate_scale_arcsec_px.value)
            if len(ps2) > 0:
                arr1[i] = 1
                arr5[i] = 1
                arr10[i] = 1
            else: ## if that failed as well, try to increase exposure time
                texp = 5
                make_image(ra_img[i],dec_img[i],bgd,seeing,texp,17,resolution,dirpath.removeprefix('./images/'),filenames[i].removesuffix('.fits'))
                ps3 = platesolve.platesolve(path.join(dirpath,filenames[i]),simulate.plate_scale_arcsec_px.value)
                if len(ps3) > 0:
                    arr5[i] = 1
                    arr10[i] = 1
                else: ## if 5 sec exposure failed, try 10 also
                    texp = 10
                    make_image(ra_img[i],dec_img[i],bgd,seeing,texp,17,resolution,dirpath.removeprefix('./images/'),filenames[i].removesuffix('.fits'))
                    ps4 = platesolve.platesolve(path.join(dirpath,filenames[i]),simulate.plate_scale_arcsec_px.value)
                    if len(ps4) > 0:
                        arr10[i] = 1
    return pd.DataFrame({'ra':ra_img,'dec':dec_img,'1_sec':arr1,'5_sec':arr5,'10_sec':arr10})

def precision_solve(dirpath,bgd,seeing,resolution,kerdir):
    ra_vec = []
    dec_vec = []
    ra_shift_vec = []
    ra_corr_vec = []
    dec_shift_vec = []
    dec_corr_vec = []
    dirs = listdir(dirpath)
    # kerdir = simulate.create_kernels(resolution,seeing * u.arcsec)

    with tqdm(total = len(dirs)) as pbar:
        for cur_dir in dirs:
            cur_path = path.join(dirpath,cur_dir)
            filenames = next(walk(cur_path), (None, None, []))[2]  # [] if no file
            n = len(filenames)

            cur_ra = []
            cur_ra_ps = []
            cur_dec = []
            cur_dec_ps = []
            ra0 = 0
            ra0_ps = 0
            dec0 = 0
            dec0_ps = 0
            for i in range(n):
                hdu = fits.open(path.join(cur_path,filenames[i]))
                cur_ra.append(hdu[0].header['RA'])
                cur_dec.append(hdu[0].header['DEC'])
                hdu.close()

                ps = platesolve.platesolve(path.join(cur_path,filenames[i]),simulate.plate_scale_arcsec_px.value)
                if len(ps) > 0:
                    cur_ra_ps.append(ps['ra_j2000_hours'] * 15)
                    cur_dec_ps.append(ps['dec_j2000_degrees'])
                else: ## if platesolve fails, take longer exposure and try again
                    make_image(cur_ra[-1],cur_dec[-1],bgd,seeing,5,16,resolution,cur_path.removeprefix('./images/'),filenames[i].removesuffix('.fits'),kerdir)
                    ps2 = platesolve.platesolve(path.join(cur_path,filenames[i]),simulate.plate_scale_arcsec_px.value)
                    if len(ps2) > 0:
                        cur_ra_ps.append(ps2['ra_j2000_hours'] * 15)
                        cur_dec_ps.append(ps2['dec_j2000_degrees'])
                    else: ## if platesolve fails again, append nan
                        cur_ra_ps.append(np.nan)
                        cur_dec_ps.append(np.nan)
                if filenames[i] == 'cntr.fits':
                    ra0 = cur_ra[-1]
                    dec0 = cur_dec[-1]
                    ra0_ps = cur_ra_ps[-1]
                    dec0_ps = cur_dec_ps[-1]
            ra_vec.append(ra0)
            dec_vec.append(dec0)
            ra_shift_vec.append(np.array(cur_ra) - ra0)
            ra_corr_vec.append(np.array(cur_ra_ps) - ra0_ps)
            dec_shift_vec.append(np.array(cur_dec) - dec0)
            dec_corr_vec.append(np.array(cur_dec_ps) - dec0_ps)
            pbar.update(1)
    # if(path.exists('./essential_files/kernels_tmp/')):
        # shutil.rmtree('./essential_files/kernels_tmp/')
    return ra_vec, ra_shift_vec, ra_corr_vec, dec_vec, dec_shift_vec, dec_corr_vec

def precision_to_csv(filename,ra_vec, ra_shift_vec, ra_corr_vec, dec_vec, dec_shift_vec, dec_corr_vec):
    ra_col = []
    dec_col = []
    ra_shift_col = []
    dec_shift_col = []
    ra_corr_col = []
    dec_corr_col = []

    for i in range(len(ra_corr_vec)):
        ra0 = ra_vec[i]
        dec0 = dec_vec[i]
        for j in range(len(ra_corr_vec[i])):
            ra_col.append(ra0)
            dec_col.append(dec0)
            ra_shift_col.append(ra_shift_vec[i][j])
            dec_shift_col.append(dec_shift_vec[i][j])
            ra_corr_col.append(ra_corr_vec[i][j])
            dec_corr_col.append(dec_corr_vec[i][j])

    df = pd.DataFrame({'ra':ra_col,'dec':dec_col,'ra_shift':ra_shift_col,
                    'dec_shift':dec_shift_col,'ra_corr':ra_corr_col,'dec_corr':dec_corr_col})
    filepath = path.join('.','ps_results',filename)
    df.to_csv(filepath,index=False)

def plot_guiding(ra_vec, ra_shift_vec, ra_corr_vec, dec_vec, dec_shift_vec, dec_corr_vec):
    fig,(ax1,ax2) = plt.subplots(1,2,figsize = (10,5),dpi=120)
    
    x1_vec = dec_shift_vec / (simulate.plate_scale_arcsec_px.value/3600)
    y1_vec = dec_corr_vec / (simulate.plate_scale_arcsec_px.value/3600)

    for x,y in zip(x1_vec,y1_vec):
        ax1.scatter(x[:10],y[:10],marker='x')
    ax1.plot(np.unique(x1_vec),np.unique(x1_vec),ls='--',color='k')
    ax1.set_yticks(np.arange(-4,4.5,0.5))
    ax1.set_xlabel('Actual shift (px)',fontdict={'size':14})
    ax1.set_ylabel('Correction (px)',fontdict={'size':14})
    ax1.set_title('plate solving for DEC correction')
    ax1.grid()
    ax1.set_ylim(-4,4)

    x2_vec = ra_shift_vec / (simulate.plate_scale_arcsec_px.value/3600)
    y2_vec = ra_corr_vec / (simulate.plate_scale_arcsec_px.value/3600)

    for x,y in zip(x2_vec,y2_vec):
        ax2.scatter(x,y,marker='x')
    ax2.plot(np.unique(x2_vec),np.unique(x2_vec),ls='--',color='k')
    ax2.set_yticks(np.arange(-4,4.5,0.5))
    ax2.set_xlabel('Actual shift (px)',fontdict={'size':14})
    ax2.set_ylabel('Correction (px)',fontdict={'size':14})
    ax2.set_title('plate solving for RA correction')
    ax2.grid()
    ax2.set_ylim(-4,4)

    plt.show()

def show_image(filepath):
    image = fits.open(path.normpath(filepath))
    fig,ax = plt.subplots(figsize=(5,5),dpi=120)
    ax.imshow(image[0].data,norm=LogNorm(),cmap='Greys_r',origin='lower')
    ax.set_xticks([])
    ax.set_yticks([])
    print(f'RA {image[0].header["RA"]}')
    print(f'DEC {image[0].header["DEC"]}')
    print(f'seeing {image[0].header["seeing"]}')
    print(f'bgd {image[0].header["bgdmagg"]}')
    mat = image[0].data
    image.close()
    return mat
    
def get_error_arrays(dataframe_lst):
    err_lst = []
    for df in dataframe_lst:
        df = df.iloc[1::2]
        dec_err_arcsec = (df['dec_corr'] - df['dec_shift']) * 3600
        ra_err_arcsec = (df['ra_corr'] - df['ra_shift']) * 3600
        err_arr = (dec_err_arcsec**2 + (ra_err_arcsec * np.cos(np.deg2rad(df['dec'])))**2 )**(1/2)
        err_lst.append(err_arr)
    return err_lst

def plot_error_hist(err_lst):
    fig,ax = plt.subplots(figsize=(5,5),dpi=120)
    ax.set_xlabel('correction error (arcsec)')
    err_arr = np.concatenate(err_lst)
    ax.hist(err_arr,bins=100,range=(-3,3),histtype='step',color='RoyalBlue',label=f'{len(err_arr)} attempts')
    txt_str = '\n'.join((
    f'50% < {scoreatpercentile(np.abs(err_arr),50):1.2f}" ',
    f'90% < {scoreatpercentile(np.abs(err_arr),90):1.2f}" ',
    f'{percentileofscore(np.abs(err_arr),1):1.0f}% < 1.00"',
    f'99% < {scoreatpercentile(np.abs(err_arr),99):1.2f}" ',))
    props = dict(boxstyle='round', facecolor='k', alpha=0.5)
    ax.text(0.05, 0.95, txt_str, transform=ax.transAxes, fontsize=12,
            verticalalignment='top', bbox=props)
    ax.legend(loc='best')
    ax.grid()
    plt.show()

def plot_error_vs_bgd(err_lst,bgd_lst):
    fig,ax = plt.subplots(figsize=(5,5),dpi=120)
    ax.set_xlabel('background (mag/arcsec^2)')
    ax.set_ylabel('correction error (arcsec)')
    mag_arr = np.unique(bgd_lst)
    arr50 = []
    arr90 = []
    arr99 = []
    for mag in mag_arr:
        err_arr = np.concatenate([err for err,bgd in zip(err_lst,bgd_lst) if bgd == mag])
        arr50.append(scoreatpercentile(np.abs(err_arr),50))
        arr90.append(scoreatpercentile(np.abs(err_arr),90))
        arr99.append(scoreatpercentile(np.abs(err_arr),99))
    ax.plot(mag_arr,arr50,label='50%',color='RoyalBlue',ls='--',marker='o',markersize=5)
    ax.plot(mag_arr,arr90,label='90%',color='Crimson',ls='--',marker='o',markersize=5)
    ax.plot(mag_arr,arr99,label='99%',color='Indigo',ls='--',marker='o',markersize=5)
    ax.set_xticks(mag_arr)
    ax.grid()
    ax.legend()
    fig.show()

def plot_error_vs_seeing(err_lst,seeing_lst):
    fig,ax = plt.subplots(figsize=(5,5),dpi=120)
    ax.set_xlabel('seeing (arcsec)')
    ax.set_ylabel('correction error (arcsec)')
    seeing_arr = np.unique(seeing_lst)
    arr50 = []
    arr90 = []
    arr99 = []
    for seeing in seeing_arr:
        err_arr = np.concatenate([err for err,fwhm in zip(err_lst,seeing_lst) if fwhm == seeing])
        arr50.append(scoreatpercentile(np.abs(err_arr),50))
        arr90.append(scoreatpercentile(np.abs(err_arr),90))
        arr99.append(scoreatpercentile(np.abs(err_arr),99))
    ax.plot(seeing_arr,arr50,label='50%',color='RoyalBlue',ls='--',marker='o',markersize=5)
    ax.plot(seeing_arr,arr90,label='90%',color='Crimson',ls='--',marker='o',markersize=5)
    ax.plot(seeing_arr,arr99,label='99%',color='Indigo',ls='--',marker='o',markersize=5)
    ax.set_xticks(seeing_arr)
    ax.grid()
    ax.legend(loc='best')
    fig.show()

def plot_fail_vs_seeing():
    seeing183_lst = []
    df183_lst = []
    filenames183 = next(walk('./ps_results/solve_vs_exp/183'), (None, None, []))[2]
    for filename in filenames183:
        df183_lst.append(pd.read_csv(path.join('./ps_results/solve_vs_exp/183',filename)))
        seeing183_lst.append(float(filename[3]))
    seeing294_lst = []
    df294_lst = []
    filenames294 = next(walk('./ps_results/solve_vs_exp/294'), (None, None, []))[2]
    for filename in filenames294:
        df294_lst.append(pd.read_csv(path.join('./ps_results/solve_vs_exp/294',filename)))
        seeing294_lst.append(float(filename[3]))

    seeing_arr = np.unique(seeing183_lst + seeing294_lst)
    
    arr_183_1, arr_183_5, arr_183_10 = [],[],[]
    arr_294_1, arr_294_5, arr_294_10 = [],[],[]

    for seeing in seeing_arr:
        df183 = pd.concat([df for df,fwhm in zip(df183_lst,seeing183_lst) if fwhm == seeing],ignore_index=True)
        df294 = pd.concat([df for df,fwhm in zip(df294_lst,seeing294_lst) if fwhm == seeing],ignore_index=True)
        arr_183_1.append(1 - df183['1_sec'].mean())
        arr_183_5.append(1 - df183['5_sec'].mean())
        arr_183_10.append(1 - df183['10_sec'].mean())
        arr_294_1.append(1 - df294['1_sec'].mean())
        arr_294_5.append(1 - df294['5_sec'].mean())
        arr_294_10.append(1 - df294['10_sec'].mean())

    fig,ax = plt.subplots(figsize=(5,5),dpi=120)
    ax.set_title('failure rate vs seeing, ASI183')
    ax.set_xlabel('seeing (arcsec)')
    ax.set_ylabel('failure rate (%)')
    ax.plot(seeing_arr,100*np.array(arr_183_1),label='1 sec exposure',color='RoyalBlue',ls='--',marker='o',markersize=5)
    ax.plot(seeing_arr,100*np.array(arr_183_5),label='5 sec exposure',color='Crimson',ls='--',marker='o',markersize=5)
    ax.plot(seeing_arr,100*np.array(arr_183_10),label='10 sec exposure',color='Indigo',ls='--',marker='o',markersize=5)
    ax.set_xticks(seeing_arr)
    ax.grid()
    ax.legend(loc='best')
    fig.show()

    fig,ax = plt.subplots(figsize=(5,5),dpi=120)
    ax.set_title('failure precentage vs seeing, ASI294')
    ax.set_xlabel('seeing (arcsec)')
    ax.set_ylabel('failure rate (%)')
    ax.plot(seeing_arr,100*np.array(arr_294_1),label='1 sec exposure',color='RoyalBlue',ls='--',marker='o',markersize=5)
    ax.plot(seeing_arr,100*np.array(arr_294_5),label='5 sec exposure',color='Crimson',ls='--',marker='o',markersize=5)
    ax.plot(seeing_arr,100*np.array(arr_294_10),label='10 sec exposure',color='Indigo',ls='--',marker='o',markersize=5)
    ax.set_xticks(seeing_arr)
    ax.grid()
    ax.legend(loc='best')
    fig.show()

def plot_fail_vs_bgd():
    bgd183_lst = []
    df183_lst = []
    filenames183 = next(walk('./ps_results/solve_vs_exp/183'), (None, None, []))[2]
    for filename in filenames183:
        df183_lst.append(pd.read_csv(path.join('./ps_results/solve_vs_exp/183',filename)))
        bgd183_lst.append(float(filename[0:2]))
    bgd294_lst = []
    df294_lst = []
    filenames294 = next(walk('./ps_results/solve_vs_exp/294'), (None, None, []))[2]
    for filename in filenames294:
        df294_lst.append(pd.read_csv(path.join('./ps_results/solve_vs_exp/294',filename)))
        bgd294_lst.append(float(filename[0:2]))

    bgd_arr = np.unique(bgd183_lst + bgd294_lst)
    
    arr_183_1, arr_183_5, arr_183_10 = [],[],[]
    arr_294_1, arr_294_5, arr_294_10 = [],[],[]

    for bgd in bgd_arr:
        df183 = pd.concat([df for df,mag in zip(df183_lst,bgd183_lst) if mag == bgd],ignore_index=True)
        df294 = pd.concat([df for df,mag in zip(df294_lst,bgd294_lst) if mag == bgd],ignore_index=True)
        arr_183_1.append(1 - df183['1_sec'].mean())
        arr_183_5.append(1 - df183['5_sec'].mean())
        arr_183_10.append(1 - df183['10_sec'].mean())
        arr_294_1.append(1 - df294['1_sec'].mean())
        arr_294_5.append(1 - df294['5_sec'].mean())
        arr_294_10.append(1 - df294['10_sec'].mean())

    fig,ax = plt.subplots(figsize=(5,5),dpi=120)
    ax.set_title('failure rate vs background, ASI183')
    ax.set_xlabel('background (mag/arcsec^2)')
    ax.set_ylabel('failure rate (%)')
    ax.plot(bgd_arr,100*np.array(arr_183_1),label='1 sec exposure',color='RoyalBlue',ls='--',marker='o',markersize=5)
    ax.plot(bgd_arr,100*np.array(arr_183_5),label='5 sec exposure',color='Crimson',ls='--',marker='o',markersize=5)
    ax.plot(bgd_arr,100*np.array(arr_183_10),label='10 sec exposure',color='Indigo',ls='--',marker='o',markersize=5)
    ax.set_xticks(bgd_arr)
    ax.grid()
    ax.legend(loc='best')
    fig.show()

    fig,ax = plt.subplots(figsize=(5,5),dpi=120)
    ax.set_title('failure precentage vs background, ASI294')
    ax.set_xlabel('background (mag/arcsec^2)')
    ax.set_ylabel('failure rate (%)')
    ax.plot(bgd_arr,100*np.array(arr_294_1),label='1 sec exposure',color='RoyalBlue',ls='--',marker='o',markersize=5)
    ax.plot(bgd_arr,100*np.array(arr_294_5),label='5 sec exposure',color='Crimson',ls='--',marker='o',markersize=5)
    ax.plot(bgd_arr,100*np.array(arr_294_10),label='10 sec exposure',color='Indigo',ls='--',marker='o',markersize=5)
    ax.set_xticks(bgd_arr)
    ax.grid()
    ax.legend(loc='best')
    fig.show()

In [7]:
## Verify image looks good

ra,dec = 271.965886, 43.983750	
bgd = 20.0
seeing = 2 * u.arcsec
texp = 10 * u.s
mag = 17
resolution = 1

make_image(ra,dec,bgd,seeing.value,texp.value,mag,resolution,'test','cntr')
im1 = show_image('./images/test/cntr.fits')
ps = platesolve.platesolve('./images/test/cntr.fits',simulate.plate_scale_arcsec_px.value)
if len(ps) > 0:
    print(f"RA {ps['ra_j2000_hours'] * 15}")
    print(f"DEC {ps['dec_j2000_degrees']}")

RA 271.965886
DEC 43.98375
seeing 2.0
bgd 20.0
RA 271.63674422141855
DEC 43.9835010251405


In [8]:
# illustration plot

# from matplotlib.patches import Rectangle

# p = simulate.plate_scale_arcsec_px.value/3600
# delta_dec = p
# shift_dec = 10 * delta_dec
# ra2 = ra 
# dec2 = dec + shift_dec
# make_image(ra2,dec2,bgd,seeing.value,texp.value,mag,resolution,'test','shift')

# im1 = show_image('./images/test/cntr.fits')
# ps = platesolve.platesolve('./images/test/cntr.fits',simulate.plate_scale_arcsec_px.value)
# im2 = show_image('./images/test/shift.fits')
# ps2 = platesolve.platesolve('./images/test/shift.fits',simulate.plate_scale_arcsec_px.value)

# fig,ax = plt.subplots(1,2,dpi=300,gridspec_kw={'height_ratios':[1],'width_ratios':[1,1]})
# ax[0].imshow(im1,norm=LogNorm(),cmap='Greys_r',origin='lower',zorder=0)
# ax[0].imshow(im2,norm=LogNorm(),cmap='Greys_r',origin='lower',alpha=0.5,zorder=10)
# ax[1].imshow(im1,norm=LogNorm(),cmap='Greys_r',origin='lower',zorder=0)
# ax[1].imshow(im2,norm=LogNorm(),cmap='Greys_r',origin='lower',alpha=0.5,zorder=10)
# i1,j1 = np.unravel_index(im1.argmax(),im1.shape)
# i2,j2 = np.unravel_index(im2.argmax(),im2.shape)
# ax[1].scatter(j1,i1,marker='.',color='r',zorder=20,alpha=0.7,s=5)
# ax[1].scatter(j2,i2,marker='.',color='r',zorder=20,alpha=0.7,s=5)
# u,v= 0.5 * (j1 - j2) ,0.5 * (i1 - i2)
# jr,ir = 0.5*(j1+j2) + 15 , 0.5*(i1+i2)
# jl,il = 0.5*(j1+j2) - 10 , 0.5*(i1+i2)
# ra_corr = (ps2['ra_j2000_hours'] - ps['ra_j2000_hours']) * 15 * 3600
# dec_corr = (ps2['dec_j2000_degrees'] - ps['dec_j2000_degrees']) * 3600
# delta_corr = (dec_corr**2 + (ra_corr * np.cos(np.deg2rad(ps['dec_j2000_degrees'])))**2)**(1/2)
# ax[1].annotate('',xy = (jl-3*u,il-3*v),xytext=(jl+3*u,il+3*v),arrowprops=dict(arrowstyle='<->',color='w',shrinkA=0.5,shrinkB=0.5),rotation=-10)
# ax[1].text(jl-77,il+15,f'real offset \n $\Delta = {10 * simulate.plate_scale_arcsec_px.value:.2f}$"',color='w',ha='center',va='center',fontsize=8)
# ax[1].text(jl-77,il-25,f'measured offset \n $\Delta = {delta_corr:.2f}$"',color='w',ha='center',va='center',fontsize=8)
# ax[0].add_patch(Rectangle((5600,2450),367,250,linewidth=1,edgecolor='r',facecolor='none'))
# ax[0].set_xticks([])
# ax[0].set_yticks([])
# ax[1].set_xticks([])
# ax[1].set_yticks([])
# ax[1].set_xlim(5555,5912)
# ax[1].set_ylim(2450,2700)
# ax[0].annotate('',xy = (0,-0.05),xytext=(1,-0.05),xycoords='axes fraction',arrowprops=dict(arrowstyle='<->'))
# ax[0].annotate('',xy = (-0.05,0),xytext=(-0.05,1),xycoords='axes fraction',arrowprops=dict(arrowstyle='<->'))
# ax[0].text(0.5,-0.1,f'{(im1.shape[1] * simulate.plate_scale_arcsec_px.value/3600):.2f}$^\circ$',ha='center',va='center',transform=ax[0].transAxes)
# ax[0].text(-0.1,0.5,f'{(im1.shape[0] * simulate.plate_scale_arcsec_px.value/3600):.2f}$^\circ$',ha='center',va='center',rotation=90,transform=ax[0].transAxes)
# ax[1].annotate('',xy = (0,-0.03),xytext=(1,-0.03),xycoords='axes fraction',arrowprops=dict(arrowstyle='<->'))
# ax[1].annotate('',xy = (1.03,0),xytext=(1.03,1),xycoords='axes fraction',arrowprops=dict(arrowstyle='<->'))
# ax[1].text(0.5,-0.1,f'{(367 * simulate.plate_scale_arcsec_px.value):.0f}"',ha='center',va='center',transform=ax[1].transAxes)
# ax[1].text(1.1,0.5,f'{(250 * simulate.plate_scale_arcsec_px.value):.0f}"',ha='center',va='center',rotation=90,transform=ax[1].transAxes)



RA 171.965886
DEC 43.98448211342593
seeing 1.0
bgd 20.0
RA 171.966019504644
DEC 43.984624509585
RA 171.965886
DEC 43.98375
seeing 1.0
bgd 20.0
RA 171.965886
DEC 43.98448211342593
seeing 1.0
bgd 20.0


Text(1.1, 0.5, '66"')

In [9]:
## Create images uniformly for timing the plate solver/checking success rate

# create_images_uniformly(300,20,3,'test/20_3')
# create_images_uniformly(300,20,2,'test/20_2')
# create_images_uniformly(300,20,1,'test/20_1')
# create_images_uniformly(300,19,3,'test/19_3')
# create_images_uniformly(300,19,2,'test/19_2')
# create_images_uniformly(300,19,1,'test/19_1')
# create_images_uniformly(300,18,3,'test/18_3')
# create_images_uniformly(300,18,2,'test/18_2')
# create_images_uniformly(300,18,1,'test/18_1')

## when plate solving, if fails, take longer exposure and try again
# ## will plot both 1 sec and 5 sec exposure results
# df1 = stats_solve('./images/test/20_3',20,3,1)
# df1.to_csv('./ps_results/solve_vs_exp/294/20_3.csv',index=False)

# df2 = stats_solve('./images/test/20_2',20,2,1)
# df2.to_csv('./ps_results/solve_vs_exp/294/20_2.csv',index=False)

# df3 = stats_solve('./images/test/20_1',20,1,1)
# df3.to_csv('./ps_results/solve_vs_exp/294/20_1.csv',index=False)

# df4 = stats_solve('./images/test/19_3',19,3,1)
# df4.to_csv('./ps_results/solve_vs_exp/294/19_3.csv',index=False)

# df5 = stats_solve('./images/test/19_2',19,2,1)
# df5.to_csv('./ps_results/solve_vs_exp/294/19_2.csv',index=False)

# df6 = stats_solve('./images/test/19_1',19,1,1)
# df6.to_csv('./ps_results/solve_vs_exp/294/19_1.csv',index=False)

# df7 = stats_solve('./images/test/18_3',18,3,1)
# df7.to_csv('./ps_results/solve_vs_exp/294/18_3.csv',index=False)

# df8 = stats_solve('./images/test/18_2',18,2,1)
# df8.to_csv('./ps_results/solve_vs_exp/294/18_2.csv',index=False)

# df9 = stats_solve('./images/test/18_1',18,1,1)
# df9.to_csv('./ps_results/solve_vs_exp/294/18_1.csv',index=False)

100%|██████████| 300/300 [22:51<00:00,  4.57s/it]


In [ ]:
## Full guiding precision test

N = 500
resolution = 5

# for seeing in [1,2,3]:
#     kerdir = simulate.create_kernels(resolution,seeing * u.arcsec)
#     for bgd in [18.5,19.5,20.5]:
#         if not path.exists(f'./ps_results/{seeing}_{bgd}.csv'):
#             makedirs(f'./images/prec_{seeing}_{bgd}',exist_ok=True)
#         create_precision_images(N,bgd,seeing,resolution,kerdir,base_dir=f'prec_{seeing}_{bgd}')
#         ra_vec, ra_shift_vec, ra_corr_vec, dec_vec, dec_shift_vec, dec_corr_vec = precision_solve(f'./images/prec_{seeing}_{bgd}',bgd,seeing,resolution,kerdir)
#         precision_to_csv(f'{seeing}_{bgd}.csv',ra_vec, ra_shift_vec, ra_corr_vec, dec_vec, dec_shift_vec, dec_corr_vec)
#         shutil.rmtree(f'./images/prec_{seeing}_{bgd}')

#     if(path.exists('./essential_files/kernels_tmp/')):
#         shutil.rmtree('./essential_files/kernels_tmp/')



In [5]:
## Read csv, plot statistics of guiding corrections

# df1 = pd.read_csv('./ps_results/1_18.5.csv')
# df2 = pd.read_csv('./ps_results/2_19.5.csv')
# df3 = pd.read_csv('./ps_results/3_20.5.csv')
# df4 = pd.read_csv('./ps_results/1_19.5.csv')
# df5 = pd.read_csv('./ps_results/2_20.5.csv')
# df6 = pd.read_csv('./ps_results/3_18.5.csv')
# df7 = pd.read_csv('./ps_results/1_20.5.csv')
# df8 = pd.read_csv('./ps_results/2_18.5.csv')
# df9 = pd.read_csv('./ps_results/3_19.5.csv')

# err_lst = get_error_arrays([df1,df2,df3,df4,df5,df6,df7,df8,df9])

# for i,err in enumerate(err_lst):
#     fail = np.count_nonzero(np.isnan(err))
#     print(f'failed to solve {fail} out of {len(err)} images')
#     err_lst[i] = err[~np.isnan(err)]
    

# plot_error_hist(err_lst)
# plot_error_vs_bgd(err_lst,[18.5,19.5,20.5,19.5,20.5,18.5,20.5,18.5,19.5])
# plot_error_vs_seeing(err_lst,[1,2,3,1,2,3,1,2,3])

failed to solve 0 out of 498 images
failed to solve 0 out of 500 images
failed to solve 3 out of 500 images
failed to solve 0 out of 496 images
failed to solve 1 out of 499 images
failed to solve 3 out of 500 images
failed to solve 0 out of 500 images
failed to solve 1 out of 499 images
failed to solve 0 out of 500 images


In [3]:
ra = []
dec = []

filenames = next(walk('./images/18-9/'), (None, None, []))[2]
for fname in tqdm(filenames,total=len(filenames)):
    ps = platesolve.platesolve(path.join('./images/18-9',fname),simulate.plate_scale_arcsec_px.value)
    ra.append(ps['ra_j2000_hours'] * 15)
    dec.append(ps['dec_j2000_degrees'])

ra = np.array(ra) * 3600
dec = np.array(dec) * 3600

100%|██████████| 12/12 [00:59<00:00,  4.99s/it]


In [4]:
seq = np.array([int(fname[4:8]) for fname in filenames])
seq = seq - seq.min() + 1
j = np.argmin(seq)

tot_error = ((ra-ra[j])**2 + ((dec-dec[j])* np.cos(np.deg2rad(dec.mean())))**2 )**(1/2)

# fig,ax = plt.subplots(dpi=300)
# # cmap = ax.scatter(ra-ra[0],dec-dec[0],c=seq)
# # cbar = fig.colorbar(cmap)
# # cbar.set_label('# of exposure')
# # ax.axvline(0,ls='--',color='k')
# ax.axhline(0,ls='--',color='k')
# cmap = ax.scatter(seq,tot_error)
# ax.set_xlabel('# of exposure')
# ax.set_ylabel('Offset from first image (arcsec)')
# ax.set_title('Acquisition exposure scatter from PS3')

In [5]:
from skimage.registration import phase_cross_correlation

im0 = fits.open(f'./images/18-9/{filenames[j]}')[0].data
tot_error_cc1 = []

for fname in tqdm(filenames,total=len(filenames)):
    im = fits.open(f'./images/18-9/{fname}')[0].data
    shift,_,_ = phase_cross_correlation(im,im0)
    shift_row = shift[0] * simulate.plate_scale_arcsec_px.value
    shift_col = shift[1] * simulate.plate_scale_arcsec_px.value
    shift_tot = (shift_row**2 + shift_col**2)**(1/2)
    tot_error_cc1.append(shift_tot)

100%|██████████| 12/12 [00:08<00:00,  1.49it/s]


In [6]:
from image_registration import chi2_shift

im0 = fits.open(f'./images/18-9/{filenames[j]}')[0].data
tot_error_cc2 = []

for fname in tqdm(filenames,total=len(filenames)):
    im = fits.open(f'./images/18-9/{fname}')[0].data
    shift_row,shift_col,_,_ = chi2_shift(im,im0)
    shift_tot = (shift_row**2 + shift_col**2)**(1/2)
    tot_error_cc2.append(shift_tot)

100%|██████████| 12/12 [00:14<00:00,  1.24s/it]


In [7]:
fig,ax = plt.subplots(dpi=300)
ax.scatter(seq,tot_error_cc2,label='CC method 2',marker='d')
ax.scatter(seq,tot_error_cc1,label='CC method 1',marker='.')
ax.scatter(seq,tot_error,label='PS3',marker='x')
ax.axhline(tot_error.std(),ls='--',color='g',label='PS3 std')
ax.set_xlabel('# of exposure')
ax.set_ylabel('$\Delta$ from first image (arcsec)')
ax.set_title('Acquisition exposure scatter')
ax.legend()

In [8]:
from matplotlib.patches import Rectangle

# Load the images with seq = 1 and seq = 2
j1 = np.argwhere(seq == 1)[0][0]
j2 = np.argwhere(seq == 3)[0][0]

im1 = fits.open(f'./images/18-9/{filenames[j1]}')[0].data
im2 = fits.open(f'./images/18-9/{filenames[j2]}')[0].data

# Perform plate solving on the images
ps1 = platesolve.platesolve(f'./images/18-9/{filenames[j1]}', simulate.plate_scale_arcsec_px.value)
ps2 = platesolve.platesolve(f'./images/18-9/{filenames[j2]}', simulate.plate_scale_arcsec_px.value)


ra_corr = (ps2['ra_j2000_hours'] - ps1['ra_j2000_hours']) * 15 * 3600
dec_corr = (ps2['dec_j2000_degrees'] - ps1['dec_j2000_degrees']) * 3600
delta_corr = (dec_corr**2 + (ra_corr * np.cos(np.deg2rad(ps1['dec_j2000_degrees'])))**2)**(1/2)

In [9]:
fig, ax = plt.subplots(1, 2, dpi=300, gridspec_kw={'height_ratios': [1], 'width_ratios': [1, 1]})
ax[0].imshow(im1, norm=LogNorm(), cmap='Greys_r', origin='lower', zorder=0)
ax[0].imshow(im2, norm=LogNorm(), cmap='Greys_r', origin='lower', alpha=0.5, zorder=10)
# ax[1].imshow(im1, norm=LogNorm(), cmap='Greys_r', origin='lower', zorder=0)
ax[1].imshow(im2/im1, norm=LogNorm(), cmap='Greys_r', origin='lower')


i1,j1 = 384,1031
i2,j2 = 378,1041


u, v = 0.5 * (j1 - j2), 0.5 * (i1 - i2)
jr, ir = 0.5 * (j1 + j2), 0.5 * (i1 + i2)
jl, il = 0.5 * (j1 + j2), 0.5 * (i1 + i2)


ax[1].annotate('', xy=(jl-u, il-v), xytext=(jl+u, il+v), arrowprops=dict(arrowstyle='<->', color='w', shrinkA=0.5, shrinkB=0.5), rotation=-10)
ax[1].text(1043, 390, f'{np.sqrt(u**2+v**2):.2f} px/{simulate.plate_scale_arcsec_px.value*np.sqrt(u**2+v**2):.2f}"', color='w', ha='center', va='center', fontsize=8)

ax[0].add_patch(Rectangle((980, 330), 80, 90, linewidth=1, edgecolor='r', facecolor='none'))
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[1].set_xticks([])
ax[1].set_yticks([])
ax[1].set_title('Image 2/Image 1, zoom in on rectangle', fontsize=8)
# ax[1].set_xlim(800, 1200)
# ax[1].set_ylim(250, 550)
ax[1].set_xlim(980, 1060)
ax[1].set_ylim(330, 420)

ax[0].annotate('', xy=(0, -0.05), xytext=(1, -0.05), xycoords='axes fraction', arrowprops=dict(arrowstyle='<->'))
ax[0].annotate('', xy=(-0.05, 0), xytext=(-0.05, 1), xycoords='axes fraction', arrowprops=dict(arrowstyle='<->'))
ax[0].text(0.5, -0.1, f'{(im1.shape[1] * simulate.plate_scale_arcsec_px.value/3600):.2f}$^\circ$', ha='center', va='center', transform=ax[0].transAxes)
ax[0].text(-0.1, 0.5, f'{(im1.shape[0] * simulate.plate_scale_arcsec_px.value/3600):.2f}$^\circ$', ha='center', va='center', rotation=90, transform=ax[0].transAxes)

ax[1].annotate('', xy=(0, -0.03), xytext=(1, -0.03), xycoords='axes fraction', arrowprops=dict(arrowstyle='<->'))
ax[1].annotate('', xy=(1.03, 0), xytext=(1.03, 1), xycoords='axes fraction', arrowprops=dict(arrowstyle='<->'))
ax[1].text(0.5, -0.1, f'{(80 * simulate.plate_scale_arcsec_px.value):.0f}"', ha='center', va='center', transform=ax[1].transAxes)
ax[1].text(1.1, 0.5, f'{(90 * simulate.plate_scale_arcsec_px.value):.0f}"', ha='center', va='center', rotation=90, transform=ax[1].transAxes)

plt.show()

/tmp/ipykernel_8592/1863086196.py:5: RuntimeWarning: invalid value encountered in divide
  ax[1].imshow(im2/im1, norm=LogNorm(), cmap='Greys_r', origin='lower')


In [27]:
fig,ax = plt.subplots(dpi=300)
j1 = np.argwhere(seq == 1)[0][0]
j2 = np.argwhere(seq == 8)[0][0]

im1 = fits.open(f'./images/18-9/{filenames[j1]}')[0].data
im2 = fits.open(f'./images/18-9/{filenames[j2]}')[0].data

ax.imshow(im2/im1, norm=LogNorm(), cmap='Greys_r', origin='lower')

/tmp/ipykernel_8592/3309711861.py:8: RuntimeWarning: invalid value encountered in divide
  ax.imshow(im2/im1, norm=LogNorm(), cmap='Greys_r', origin='lower')
